# Automatically ver

- Download Genbank file
- Make Genbank file to fasta file
- Alignment with Prank

<br />

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd /content/gdrive/My\ Drive/PGL/study/2019_project_chloroplast

/content/gdrive/My Drive/PGL/study/2019_project_chloroplast


In [0]:
# 다운받은 genbank file 모두 불러올때 사용
import glob

# Genbank file 불러올때 사용
! pip3 install biopython
from Bio import SeqIO

# Fasta file 만들 때 사용 
import pandas as pd
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# DataFrame Merge
from functools import reduce

# Prank for alignment
! apt-get install prank

     |████████████████████████████████| 2.2MB 4.6MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  prank
0 upgraded, 1 newly installed, 0 to remove and 16 not upgraded.
Need to get 379 kB of archives.
After this operation, 1,123 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 prank amd64 0.0.170427+dfsg-1 [379 kB]
Fetched 379 kB in 1s (332 kB/s)
Selecting previously unselected package prank.
(Reading database ... 130942 files and directories currently installed.)
Preparing to unpack .../prank_0.0.170427+dfsg-1_amd64.deb ...
Unpacking prank (0.0.170427+dfsg-1) ...
Setting up prank (0.0.170427+dfsg-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


**Download Genbank file** <br />
https://www.reddit.com/r/bioinformatics/comments/5h4qnc/what_is_the_best_way_to_download_genbank_data/ <br />
https://www.ncbi.nlm.nih.gov/books/NBK25500/#chapter1.Downloading_Full_Records <br />
http://metag-crash-course.readthedocs.io/en/latest/fetch-data/ (google : linux get genbank id) <br />
https://www.lesstif.com/pages/viewpage.action?pageId=14745703 (google : linux curl)

<br />
++TEST++

In [0]:
i = 'DQ400350.1'
! curl -s  "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=${i}&rettype=gb&retmode=txt">$i.gb

# ref 첫번째 사이트의 코드 그대로 사용시 이름이 지정되지 않은 '.gb'라는 파일이 생김
# '.gb' 파일 내용을 보니 Supplied id parameter is empty. 라는 내용 뿐

In [0]:
! curl -s  "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=${i}&rettype=gb&retmode=txt"

# 뒤의 이름을 변경해주는 코드를 빼고 시도, id의 '.1' 부분을 인식못함

 Error: CEFetchPApplication::proxy_stream(): Failed to understand id: .1



In [0]:
i = 'JN160605_1'
! curl -s  "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=${i}&rettype=gb&retmode=txt"

# '.1'이 없는 id를 사용해보아도 같은 error

Supplied id parameter is empty.


In [0]:
i = 'DQ400350.1'
! curl "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=${i}&rettype=gb&retmode=txt">$i.gb

# ref 네번째 사이트를 참조해, curl -s는 아래의 진행내역을 보여주지 않는 명령어임을 알게되어 빼봄
# 결과로 error창을 띄우지 않은 첫번째 코드를 사용

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    32    0    32    0     0    117      0 --:--:-- --:--:-- --:--:--   117


In [0]:
! curl "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nuccore&id=JN160605.1&rettype=gb&retmode=text" > JN160605_1.gb

# 변수를 사용하지 않아봄

# 위와 비교해보니 파일 용량도 알 수 있음을 알게됨
# 정상적으로 download 완료되었음

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  289k    0  289k    0     0   603k      0 --:--:-- --:--:-- --:--:--  602k


In [0]:
i = 'JN160604.1'

! curl "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nuccore&id=${i}&rettype=gb&retmode=text" >${i}.gb

# .1.gb 라는 이름으로 file 생성
# 파일 내용을 보니 Error: CEFetchPApplication::proxy_stream(): Failed to understand id: .1 라는 내용 뿐

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    32    0    32    0     0    256      0 --:--:-- --:--:-- --:--:--   256


In [0]:
! curl "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nuccore&id={i}&rettype=gb&retmode=text" >${i}.gb

# 주소부분의 id에 $를 뺐더니 .1.gb라는 이름의 file 생성, 하지만 내용은 내가 원하는 genbank file format이 맞음
# $부분이 문제였음을 알 수 있음

In [0]:
! curl "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nuccore&id={i}&rettype=gb&retmode=text" >{i}.gb

# JN160604.1 이름으로 file 생성, 내용 또한 genbank file format 맞음

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  290k    0  290k    0     0   579k      0 --:--:-- --:--:-- --:--:--  579k


--TEST--
<br /><br /><br />
++APPLICATION++

In [0]:
gb_id_list = ['NC_015899.1', 'NC_010109.1', 'NC_015891.1', 'NC_015894.1', 'NC_033970.1', 'NC_027954.2', 'NC_027272.1', 'KY769273.1', 'NC_027681.1', 'NC_035499.1', 'KR296655.1', 'X86563.2']

for i in gb_id_list :
  ! curl "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nuccore&id={i}&rettype=gb&retmode=text" >gb_file/{i}.gb
  
# output 부분에 원하는 폴더내에 다운받고 싶다면 그 폴더를 만들어주고 root를 지정해주어도 됨

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  315k    0  315k    0     0   598k      0 --:--:-- --:--:-- --:--:--  598k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  311k    0  311k    0     0   632k      0 --:--:-- --:--:-- --:--:--  631k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  319k    0  319k    0     0   686k      0 --:--:-- --:--:-- --:--:--  686k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  315k    0  315k    0     0   644k      0 --:--:-- --:--:-- --:--:--  643k
  % Total    % Received % Xferd  Average Speed   Tim

--APPLICATION--
<br /><br /><br />
**Make Genbank file to fasta file**

In [0]:
gb_glob_list = glob.glob('./gb_file/*.gb')

In [0]:
gb_glob_list

['./gb_file/NC_015899.1.gb',
 './gb_file/NC_010109.1.gb',
 './gb_file/NC_015891.1.gb',
 './gb_file/NC_015894.1.gb',
 './gb_file/NC_033970.1.gb',
 './gb_file/NC_027954.2.gb',
 './gb_file/NC_027272.1.gb',
 './gb_file/KY769273.1.gb',
 './gb_file/NC_027681.1.gb',
 './gb_file/NC_035499.1.gb',
 './gb_file/KR296655.1.gb',
 './gb_file/X86563.2.gb']

In [0]:
gb_parse_list = [[file for file in SeqIO.parse(x, 'genbank')] for x in gb_glob_list]

In [0]:
gb_parse_list

[[SeqRecord(seq=Seq('AAATAGCTAAATTAAATATTAAATTGTGATATAAAATTCTATTATTACATTAAG...TGT', IUPACAmbiguousDNA()), id='NC_015899.1', name='NC_015899', description='Wolffia australiana chloroplast, complete genome', dbxrefs=['Project:71217', 'BioProject:PRJNA71217'])],
 [SeqRecord(seq=Seq('TTAGATTTATATTATTAGATTTATCTAATATATTTTATATAATTAATTATCTAA...CAC', IUPACAmbiguousDNA()), id='NC_010109.1', name='NC_010109', description='Lemna minor chloroplast, complete genome', dbxrefs=['Project:28017', 'BioProject:PRJNA28017'])],
 [SeqRecord(seq=Seq('ATATTGAAAATAGCATTTTTGAAATTTTGGAAATATTAAATAATTAAATATAAT...GGC', IUPACAmbiguousDNA()), id='NC_015891.1', name='NC_015891', description='Spirodela polyrhiza chloroplast, complete genome', dbxrefs=['BioProject:PRJNA71203'])],
 [SeqRecord(seq=Seq('AAGAATCTAAGAATTATTACTCGATAGAATTCCCAATATGTAGAACAATAATAA...TTT', IUPACAmbiguousDNA()), id='NC_015894.1', name='NC_015894', description='Wolffiella lingulata chloroplast, complete genome', dbxrefs=['Project:71207', 'BioProject:

In [0]:
for gb_parse in gb_parse_list :
  print(gb_parse)

[SeqRecord(seq=Seq('AAATAGCTAAATTAAATATTAAATTGTGATATAAAATTCTATTATTACATTAAG...TGT', IUPACAmbiguousDNA()), id='NC_015899.1', name='NC_015899', description='Wolffia australiana chloroplast, complete genome', dbxrefs=['Project:71217', 'BioProject:PRJNA71217'])]
[SeqRecord(seq=Seq('TTAGATTTATATTATTAGATTTATCTAATATATTTTATATAATTAATTATCTAA...CAC', IUPACAmbiguousDNA()), id='NC_010109.1', name='NC_010109', description='Lemna minor chloroplast, complete genome', dbxrefs=['Project:28017', 'BioProject:PRJNA28017'])]
[SeqRecord(seq=Seq('ATATTGAAAATAGCATTTTTGAAATTTTGGAAATATTAAATAATTAAATATAAT...GGC', IUPACAmbiguousDNA()), id='NC_015891.1', name='NC_015891', description='Spirodela polyrhiza chloroplast, complete genome', dbxrefs=['BioProject:PRJNA71203'])]
[SeqRecord(seq=Seq('AAGAATCTAAGAATTATTACTCGATAGAATTCCCAATATGTAGAACAATAATAA...TTT', IUPACAmbiguousDNA()), id='NC_015894.1', name='NC_015894', description='Wolffiella lingulata chloroplast, complete genome', dbxrefs=['Project:71207', 'BioProject:PRJNA71

In [0]:
for gb_parse in gb_parse_list :
  for gb in gb_parse :
    print(gb.features[0:5])
    
# glob로 genbank file 모두를 가져온 것이므로 for문을 한 번 더 돌려야 다룰 수 있음

[SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(168704), strand=1), type='source'), SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(91454), strand=1), type='misc_feature'), SeqFeature(FeatureLocation(ExactPosition(111), ExactPosition(186), strand=-1), type='gene'), SeqFeature(FeatureLocation(ExactPosition(111), ExactPosition(186), strand=-1), type='tRNA'), SeqFeature(FeatureLocation(ExactPosition(379), ExactPosition(1441), strand=-1), type='gene')]
[SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(165955), strand=1), type='source'), SeqFeature(CompoundLocation([FeatureLocation(ExactPosition(74637), ExactPosition(74751), strand=-1), FeatureLocation(ExactPosition(103412), ExactPosition(104210), strand=-1)], 'join'), type='gene', location_operator='join'), SeqFeature(CompoundLocation([FeatureLocation(ExactPosition(74637), ExactPosition(74751), strand=-1), FeatureLocation(ExactPosition(103978), ExactPosition(104210), strand=-1), FeatureLocation(ExactPositio

In [0]:
for gb_parse in gb_parse_list :
  for gb in gb_parse :
    print(gb.annotations['organism'])

Wolffia australiana
Lemna minor
Spirodela polyrhiza
Wolffiella lingulata
Symplocarpus renifolius
Epipremnum aureum
Dieffenbachia seguine
Colocasia esculenta
Pinellia ternata
Zantedeschia aethiopica
Alocasia macrorrhizos
Zea mays


In [0]:
name_list = []
for gb_parse in gb_parse_list :
  for gb in gb_parse :
    name_list.append(gb.annotations['organism'])

In [0]:
name_list

['Wolffia australiana',
 'Lemna minor',
 'Spirodela polyrhiza',
 'Wolffiella lingulata',
 'Symplocarpus renifolius',
 'Epipremnum aureum',
 'Dieffenbachia seguine',
 'Colocasia esculenta',
 'Pinellia ternata',
 'Zantedeschia aethiopica',
 'Alocasia macrorrhizos',
 'Zea mays']

In [0]:
name = [[gb.annotations['organism'] for gb in gb_parse] for gb_parse in gb_parse_list]

In [0]:
name

[['Wolffia australiana'],
 ['Lemna minor'],
 ['Spirodela polyrhiza'],
 ['Wolffiella lingulata'],
 ['Symplocarpus renifolius'],
 ['Epipremnum aureum'],
 ['Dieffenbachia seguine'],
 ['Colocasia esculenta'],
 ['Pinellia ternata'],
 ['Zantedeschia aethiopica'],
 ['Alocasia macrorrhizos'],
 ['Zea mays']]

In [0]:
sum(name, [])

['Wolffia australiana',
 'Lemna minor',
 'Spirodela polyrhiza',
 'Wolffiella lingulata',
 'Symplocarpus renifolius',
 'Epipremnum aureum',
 'Dieffenbachia seguine',
 'Colocasia esculenta',
 'Pinellia ternata',
 'Zantedeschia aethiopica',
 'Alocasia macrorrhizos',
 'Zea mays']

In [0]:
name = sum([[gb.annotations['organism'] for gb in gb_parse] for gb_parse in gb_parse_list], [])

In [0]:
name

['Wolffia australiana',
 'Lemna minor',
 'Spirodela polyrhiza',
 'Wolffiella lingulata',
 'Symplocarpus renifolius',
 'Epipremnum aureum',
 'Dieffenbachia seguine',
 'Colocasia esculenta',
 'Pinellia ternata',
 'Zantedeschia aethiopica',
 'Alocasia macrorrhizos',
 'Zea mays']

In [0]:
# genbank file을 glob하는 것부터 dataframe을 만들기까지

def make_df_without_IR(all_genbank_file_root) :
    
    gb_glob_list = glob.glob(all_genbank_file_root)
    gb_parse_list = [[file for file in SeqIO.parse(x, 'genbank')] for x in gb_glob_list]
    
    for gb_parse in gb_parse_list :
      CDS_dic = {}
      IR_list = []
      for gb in gb_parse :
        name = gb.annotations['organism']
        for feature in gb.features :
          if feature.type == 'CDS' :
            try :
              for gene in feature.qualifiers['gene'] :
                if gene in CDS_dic :
                  IR_list.append(gene)
                else :
                  CDS_dic[gene] = str(feature.location.extract(parent_sequence = gb.seq).translate(table=11, to_stop=True))
            except KeyError :
              pass
                        
        for IR in set(IR_list) :
          del CDS_dic[IR]

        df_temporary_CDS = pd.DataFrame(list(CDS_dic.items()), columns=['Gene name', '%s'%name])
        
        try :
          df_CDS = reduce(lambda left,right: pd.merge(left,right,on='Gene name'), [df_CDS, df_temporary_CDS])
        except NameError :
          df_CDS = df_temporary_CDS
          
    return(df_CDS)

In [0]:
make_df_without_IR('./gb_file/*.gb').shape

(53, 13)

In [0]:
make_df_without_IR('./gb_file/*.gb')

,Gene name,Wolffia australiana,Lemna minor,Spirodela polyrhiza,Wolffiella lingulata,Symplocarpus renifolius,Epipremnum aureum,Dieffenbachia seguine,Colocasia esculenta,Pinellia ternata,Zantedeschia aethiopica,Alocasia macrorrhizos,Zea mays
0,psbA,MTAILERRESTNLWARFCNWVTSTENRLYIGWFGVLMIPTLLTATS...,MTAILERRESTSLWGRFCNWVTSTENRLYIGWFGVLMIPTLLTATS...,MTAILERRESTSLWGRFCNWVTSTENRLYIGWFGVLMIPTLLTATS...,MTAILERRESTNLWARFCNWVTSTENRLYIGWFGVLMIPTLLTATS...,MTAILERRESTSLWGRFCNWITSTENRLYIGWFGVLMIPTLLTATS...,MTAILERRESTSLWGRFCNWITSTENRLYIGWFGVLMIPTLLTATS...,MTAILERRESTSLWGRFCNWITSTENRLYIGWFGVLMIPTLLTATS...,MTAILERRESTSLWGRFCNWITSTENRLYIGWFGVLMIPTLLTATS...,MTAILERRESTSLWGRFCNWITSTENRLYIGWFGVLMIPTLLTATS...,MTAILERRESTSLWGRFCNWITSTENRLYIGWFGVLMIPTLLTATS...,MTAILERRESTSLWGRFCNWITSTENRLYIGWFGVLMIPTLLTATS...,MTAILERRESTSLWGRFCNWITSTENRLYIGWFGVLMIPTLLTATS...
1,psbK,MLNIFLNLICICLNSAVSSSSSVFLGKLPEAYAIFNPLIDILPIIP...,MLNPFFNLICICLNSALSSSGVFFGKLPEAYAIFNPIVDIMPVIPV...,MLNIFLNLICICLNSVLSSSSFFFGKLPEAYAIFNPIVDIMPVIPL...,MLNIFLNLICICLNSAVSSSSVFLGKLPEAYAIFNPIIDIMPVIPL...,MLNIFSLICICINSALYSSSFFFGKLPEAYALFNPIVDVMPVIPVL...,MLNIFFSLICICFNSILSSSSFFFGKLPEAYALFNPIIDIMPVIPV...,MLNILFSLICICLNSALSSSSFFFGKLPEAYALFNPIIDIMPVIPV...,MLNIFFSLICICLDSALSSSSFFFGKLPEAYALFNPIVDIMPVIPV...,MLNIFFSLICICLNSALSSSSFFFGKLPEAYALFNPIVDIMPVIPV...,MLNIFFSLICICLNSALSSSSFFFGKLPEAYALFNPIVDIMPVIPV...,MFNIFSLICICLNSTPYSSSFFFAKLPEAYAFFNPIVDVMPIIPLL...,MPNILSLTCICFNSVLCPTSFFFAKLPEAYAIFNPIVDVMPVIPVL...
2,psbI,MLTLKLFVYTVVIFFVSLFIFGFLSNDPGRNPGREE,MLTLKLFVYTVVIFFVSLFIFGFLSNDPGRNPGREE,MLTLKLFVYTVVIFFVSLFIFGFLSNDPGRNPGREE,MLTLKLFVYTVVIFFVSLFIFGFLSNDPGRNPGREE,MLTLKLFVYTVVIFFVSLFIFGFLSNDPGRNPGREE,MLTLKPFVYTVVIFFVSLFIFGFLSNDPGRNPGREEKKKTK,MLTLKLFVYTVVIFFVSLFIFGFLSNDPGRNPGREE,MLTLKLFVYTVVIFFVSLFIFGFLSNDPGRNPGREE,MLTLKLFVYTVVIFFVSLFIFGFLSNDPGRNPGREE,MLTLKLFVYTVVIFFVSLFIFGFLSNDPGRNPGREE,MLTLKLFVYTVVIFFVSLFIFGFLSNDPGRNPGREE,MLTLKLFVYTVVIFFVSLFIFGFLSNDPGRNPGREE
3,atpA,MVTIRADEISSIIRERIEQYTREVKVVNTGTVLQVGDGIARVHGLD...,MVTIRADEISSIIRERIEQYTREVKVVNTGTVLQVGDGIARVHGLD...,MVTIRADEISNIIRERIEQYNREVKVVNTGTVLQVGDGIARVHGLD...,MVTIRADEISSIIRERIEQYTREVKVVNTGTVLQVGDGIARVHGLD...,MVTIRADEISNIIRERIEQYNREVKVVNTGTVLQVGDGIARIHGLD...,MENLGADEINNIIRQRIELYKIEVKVTDTGTVLRVGDGIARVCGLH...,MVTIRADEISNIIRERIEQYNREVKVVNTGTVLQVGDGIARIYGLD...,MVTLRADEISNIIRDRIEQYNREVKVVNTGTVLQVGDGIARIHGLD...,MVTIRADEISNIIRERIEQYNREVKVVNTGTVLQVGDGIARIHGLD...,MVTIRADEISNIIRERIEQYNREVKVVNTGTVLQVGDGIARIHGLD...,MGTLRADEISNIIRERIEQYNREVKIVNTGTVLQVGDGIARIHGLD...,MATLRVDEINKILRERIEQYNRKVGIENIGRVVQVGDGIARIIGLG...
4,atpF,MKNVTDSFVSLGHWPSAGGFGFNTDIFATNPINLSVVIGVLFYFGK...,MQNVTDSFVSLGHWPSAGGFGFNTDILATNPINLSVVLGVVIYFGK...,MKNVTDSFVSLGHWPSAGGFGFNTDILATNPINLSVVLGVLIYFGK...,MKNVTDSFVSLGHWPSAGGFGFNTDIFATNPINLSVVIGVVIYFGK...,MKNVTDSFVSLGHWPSAGILGFNTDILATNPINLSVVLGVLIYFGK...,MKSVTDSFVSLGHWPSAGGFGFNTDILATNPINLSVVLGVLIYFGK...,MKSVTDSFVSLGHWPSAGGLGFNTDILATNPINLSVVLGVLIYFGK...,MKSVTDSFVSLGHWPSAGGFGFNTDILATNPINLSAVLGVLIYFGK...,MKSVTDSFVSLGHWPSAGGFGFNTDILATNPINLSVVLGVLIYFGK...,MKSVTDSFVSLGHWPYAGGLGFNTDILATNPINLSVVLGVLIYFGK...,MKNLIDSFVSVGHWSSAGSFEFNTDILATNPINLSVVLGVLIFFGK...,MKNVTHSFVFLAHWPFAGSFGLNTDILATNLINLTVVVGVLIFFGK...
5,atpH,MNPLISAASVIAAGLAVGLASIGPGVGQGTAAGQAVEGIARQPEAE...,MNPLISAASVIAAGLAVGLASIGPGVGQGTAAGQAVEGIARQPEAE...,MNPLISAASVIAAGLAVGLASIGPGVGQGTAAGQAVEGIARQPEAE...,MNPLISAASVIAAGLAVGLASIGPGVGQGTAAGQAVEGIARQPEAE...,MNPLISAASVIAAGLAVGLASIGPGIGQGTAAGQAVEGIARQPEAE...,MNPLISAASVIAAGLAVGLASIGPGIGQGTAAGQALEGISRQPEAE...,MNPLISAASVIAAGLAVGLASIGPGVGQGTAAGQAVEGIARQPEAE...,MNPLISAASVIAAGLAVGLASIGPGIGQGTAAGQAVEGIARQPEAE...,MNPLISAASVIAAGLAVGLASIGPGVGQGTAAGQAVEGIARQPEAE...,MNPLISAASVIAAGLAVGLASIGPGVGQGTAAGQAVEGIARQPEAE...,MNPLISAASVIAAGLAVGLASIGPGVGQGTAAGQAVEGIARQPEAE...,MNPLIAAASVIAAGLAVGLASIGPGVGQGTAAGQAVEGIARQPEAE...
6,atpI,MNVIPCSINTLKGLYDISGVEVGQHLYWQIGGLQVHAQVLITSWVV...,MNVIPCSINTLKGLYDISGVEVGQHLYWQIGGLQVHAQVLITSWVV...,MNVIPCSINTLTGLYDISGVEVGQHLYWQIGGLQVHAQVLITSWVV...,MNVIPCSINTLNGLYDISGVEVGQHLYWQIGGLQVHAQVLITSWVV...,MNLIPCSINTLKGLYDISGVEVGQHLYWQRGGLQVHAQVLITSWVV...,MNVIPCSINTLK

```
def make_fasta_without_IR(all_genbank_file_root) :
    
    glob_list = glob.glob(all_genbank_file_root)
    gb_parse_list = [[file for file in SeqIO.parse(x, 'genbank')] for x in glob_list]
    
    for gb_parse in gb_parse_list :
      CDS_dic = {}
      IR_list = []
      for gb in gb_parse :
        name = gb.annotations['organism']
        for feature in gb.features :
          if feature.type == 'CDS' :
            try :
              for gene in feature.qualifiers['gene'] :
                if gene in CDS_dic :
                  IR_list.append(gene)
                else :
                  CDS_dic[gene] = str(feature.location.extract(parent_sequence = gb.seq).translate(table=11, to_stop=True))
            except KeyError :
              pass
                        
        for IR in set(IR_list) :
          del CDS_dic[IR]

        df_temporary_CDS = pd.DataFrame(list(CDS_dic.items()), columns=['Gene name', '%s'%name])
        
        try :
          df_CDS = reduce(lambda left,right: pd.merge(left,right,on='Gene name'), [df_CDS, df_temporary_CDS])
        except NameError :
          df_CDS = df_temporary_CDS
    df_CDS_ix = df_CDS.set_index('Gene name')
    for ix in list(df_CDS_ix.index) :
      proteins = (SeqRecord(Seq(df_CDS_ix[i].loc[ix]), i) for i in df_CDS_ix)
      SeqIO.write(proteins, './fasta_file/%s_fasta.fasta'%ix, 'fasta')
```
      
이 코드로 fasta file을 만드니 fasta file내에

```
>Wolffia australiana <unknown description>
MVTIRADEISSIIRERIEQYTREVKVVNTGTVLQVGDGIARVHGLDKVMAGELVEFEEGT...
```
  
처럼 종 이름 뒤에 ```<unknown description>``` 이 생겨남 <br />
이 fasta file을 이용해 prank를 돌리니
  
```
>Wolffia_australiana_<unknown_description>
MVTIRADEISSIIRERIEQYTREVKVVNTGTVLQVGDGIARVHGLDKVMAGELVEFEEGT...
```
prank 결과 file인 fas file에도 이처럼 종 이름 뒤에 붙어 존재함 <br />
이 prank 결과 file인 fas file을 이용해 beast를 돌리려하니

```
Loading additional development parsers from development_parsers.properties, which is additional set of parsers only available for development version ...
Parsing XML file: every_fas.xml
  File encoding: MS949
Looking for plugins in C:\PGL\not_limforest\Genomics\PGL\4_Beast\BEAST.v1.10.4\BEAST v1.10.4\plugins
Fatal error: 
	Line number: 15
	Column number: 23
	Error message: The value of attribute "id" associated with an element type "taxon" must not contain the '<' character.

BEAST has terminated with an error. Please select QUIT from the menu.
Jun 26, 2019 1:55:59 PM dr.app.beast.BeastMain <init>
SEVERE: Error running file: every_fas.xml
Jun 26, 2019 1:55:59 PM dr.app.beast.BeastMain <init>
SEVERE: Parsing error - poorly formed XML (possibly not an XML file):
The value of attribute "id" associated with an element type "taxon" must not contain the '<' character.
```
이와 같은 error가 생겼다. <br />
Error message를 통해 fas file 내에 있는 종 이름 뒤의 ```<unknown description>```에 의한 error임을 알 수 있음<br />
때문에 description을 없애는 코드로 다시 시도 (fasta file 만드는 코드에 description="" 부분을 추가로 처리) (https://github.com/biopython/biopython/issues/831)

In [0]:
def make_fasta_without_IR(all_genbank_file_root) :
    
    glob_list = glob.glob(all_genbank_file_root)
    gb_parse_list = [[file for file in SeqIO.parse(x, 'genbank')] for x in glob_list]
    
    for gb_parse in gb_parse_list :
      CDS_dic = {}
      IR_list = []
      for gb in gb_parse :
        name = gb.annotations['organism']
        for feature in gb.features :
          if feature.type == 'CDS' :
            try :
              for gene in feature.qualifiers['gene'] :
                if gene in CDS_dic :
                  IR_list.append(gene)
                else :
                  CDS_dic[gene] = str(feature.location.extract(parent_sequence = gb.seq).translate(table=11, to_stop=True))
            except KeyError :
              pass
                        
        for IR in set(IR_list) :
          del CDS_dic[IR]

        df_temporary_CDS = pd.DataFrame(list(CDS_dic.items()), columns=['Gene name', '%s'%name])
        
        try :
          df_CDS = reduce(lambda left,right: pd.merge(left,right,on='Gene name'), [df_CDS, df_temporary_CDS])
        except NameError :
          df_CDS = df_temporary_CDS
    df_CDS_ix = df_CDS.set_index('Gene name')
    for ix in list(df_CDS_ix.index) :
      proteins = (SeqRecord(Seq(df_CDS_ix[i].loc[ix]), i, description="") for i in df_CDS_ix)
      SeqIO.write(proteins, './fasta_file/%s_fasta.fasta'%ix, 'fasta')

In [0]:
make_fasta_without_IR('./gb_file/*.gb')

# fasta file 53개 생성

**Alignment with Prank** <br />
https://www.howtoinstall.co/en/ubuntu/trusty/prank

In [0]:
fasta_glob_list = glob.glob('./fasta_file/*.fasta')

In [0]:
fasta_glob_list

['./fasta_file/psbA_fasta.fasta',
 './fasta_file/psbK_fasta.fasta',
 './fasta_file/psbI_fasta.fasta',
 './fasta_file/atpA_fasta.fasta',
 './fasta_file/atpF_fasta.fasta',
 './fasta_file/atpH_fasta.fasta',
 './fasta_file/atpI_fasta.fasta',
 './fasta_file/psbM_fasta.fasta',
 './fasta_file/psbD_fasta.fasta',
 './fasta_file/psbC_fasta.fasta',
 './fasta_file/psaB_fasta.fasta',
 './fasta_file/psaA_fasta.fasta',
 './fasta_file/ndhJ_fasta.fasta',
 './fasta_file/ndhK_fasta.fasta',
 './fasta_file/ndhC_fasta.fasta',
 './fasta_file/atpE_fasta.fasta',
 './fasta_file/atpB_fasta.fasta',
 './fasta_file/rbcL_fasta.fasta',
 './fasta_file/psaI_fasta.fasta',
 './fasta_file/cemA_fasta.fasta',
 './fasta_file/petA_fasta.fasta',
 './fasta_file/psbJ_fasta.fasta',
 './fasta_file/psbL_fasta.fasta',
 './fasta_file/psbF_fasta.fasta',
 './fasta_file/psbE_fasta.fasta',
 './fasta_file/psaJ_fasta.fasta',
 './fasta_file/rpl20_fasta.fasta',
 './fasta_file/clpP_fasta.fasta',
 './fasta_file/psbB_fasta.fasta',
 './fasta_fil

In [0]:
len(fasta_glob_list)

53

In [0]:
for x in fasta_glob_list :
  print(x.split('/')[-1])

psbA_fasta.fasta
psbK_fasta.fasta
psbI_fasta.fasta
atpA_fasta.fasta
atpF_fasta.fasta
atpH_fasta.fasta
atpI_fasta.fasta
psbM_fasta.fasta
psbD_fasta.fasta
psbC_fasta.fasta
psaB_fasta.fasta
psaA_fasta.fasta
ndhJ_fasta.fasta
ndhK_fasta.fasta
ndhC_fasta.fasta
atpE_fasta.fasta
atpB_fasta.fasta
rbcL_fasta.fasta
psaI_fasta.fasta
cemA_fasta.fasta
petA_fasta.fasta
psbJ_fasta.fasta
psbL_fasta.fasta
psbF_fasta.fasta
psbE_fasta.fasta
psaJ_fasta.fasta
rpl20_fasta.fasta
clpP_fasta.fasta
psbB_fasta.fasta
psbN_fasta.fasta
psbH_fasta.fasta
petB_fasta.fasta
petD_fasta.fasta
rpl14_fasta.fasta
rpl22_fasta.fasta
ndhF_fasta.fasta
ndhE_fasta.fasta
ndhG_fasta.fasta
ndhA_fasta.fasta
ndhH_fasta.fasta
rps2_fasta.fasta
rpoC2_fasta.fasta
rpoB_fasta.fasta
rps14_fasta.fasta
rps4_fasta.fasta
rpl33_fasta.fasta
rps18_fasta.fasta
rpoA_fasta.fasta
rps11_fasta.fasta
rpl36_fasta.fasta
rps8_fasta.fasta
rps3_fasta.fasta
rpl32_fasta.fasta


In [0]:
# Prank for alignment
! apt-get install prank

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  prank
0 upgraded, 1 newly installed, 0 to remove and 4 not upgraded.
Need to get 379 kB of archives.
After this operation, 1,123 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 prank amd64 0.0.170427+dfsg-1 [379 kB]
Fetched 379 kB in 1s (588 kB/s)
Selecting previously unselected package prank.
(Reading database ... 130963 files and directories currently installed.)
Preparing to unpack .../prank_0.0.170427+dfsg-1_amd64.deb ...
Unpacking prank (0.0.170427+dfsg-1) ...
Setting up prank (0.0.170427+dfsg-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [0]:
for i in fasta_glob_list :
  input_name = i
  output_name = i.split('/')[-1].split('_')[0]
  ! prank -d={input_name} -o=./prank_file/{output_name} -showxml -showtree 
  
  
# xml과 dnd도 만듦 
# -showxml, -showtree 부분을 빼면 fas file만 만들어 줄 것
# prank -d={input_name} -o=./prank_file/{output_name}


-----------------
 PRANK v.170427:
-----------------

Input for the analysis
 - aligning sequences in '../fasta_file/psbA_fasta.fasta'
 - using inferred alignment guide tree
 - option '+F' is not used; it can be enabled with '+F'


Generating approximate guide tree.
aligning 0 to 1 (1%)                    aligning 0 to 2 (3%)                    aligning 0 to 3 (4%)                    aligning 0 to 4 (6%)                    aligning 0 to 5 (7%)                    aligning 0 to 6 (9%)                    aligning 0 to 7 (10%)                    aligning 0 to 8 (12%)                    aligning 0 to 9 (13%)                    aligni

In [0]:
# xml과 dnd도 만들어 섞여져있으므로 alignment 된 fas file만 따로 골라 옮겨줌

mkdir complete_alignment_fas_only

In [0]:
cp ./prank_file/*.fas ./complete_alignment_fas_only/

---